In [1]:
import torch
import gc
import os
import numpy as np
import pandas as pd
import joblib
from torch import nn
from torch.utils.data import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from peft import PeftModel, PeftConfig # ⭐️ [수정] PeftConfig 임포트
from sklearn.preprocessing import StandardScaler
from safetensors.torch import load_file

print("--- [1] 라이브러리 임포트 완료 ---")



BASE_MODEL_PATH = "/kaggle/input/debertav3large"
SAVED_MODEL_PATH = "/kaggle/input/real-jonna-last/transformers/default/1" 
TEST_CSV_PATH = "/kaggle/input/llm-classification-finetuning/test.csv"
VADER_PATH = "/kaggle/input/vader/transformers/default/1/vader_model.pkl"
MAX_LEN = 512
BATCH_SIZE = 16 
BIAS_DIM = 6


def style_counts(text):
    text = str(text) 
    ex = text.count("!")
    qm = text.count("?")
    co = text.count(",")
    return ex, qm, co

class HybridClassifier(nn.Module):
    def __init__(self, backbone, bias_dim, num_labels=3):
        super().__init__()
        self.backbone = backbone 
        classifier_in_features = self.backbone.classifier.in_features 
        self.bias_proj = nn.Sequential(
            nn.Linear(bias_dim, 64),
            nn.ReLU(),
            nn.Dropout(0.1)
        )
        self.classifier = nn.Linear(classifier_in_features + 64, num_labels)
        self.backbone.classifier = nn.Identity() 

    def forward(self, input_ids=None, attention_mask=None, bias_features=None, labels=None):
        outputs = self.backbone(input_ids=input_ids, attention_mask=attention_mask)
        cls_emb = outputs.logits 
        bias_vec = self.bias_proj(bias_features)
        fused = torch.cat([cls_emb, bias_vec], dim=1) 
        logits = self.classifier(fused) 
        loss = None
        if labels is not None:
            loss = nn.CrossEntropyLoss()(logits, labels)
        return {"loss": loss, "logits": logits}

print("--- [2] 훈련용 클래스/함수 재정의 완료 ---")


device = "cuda" if torch.cuda.is_available() else "cpu"

# 1. 토크나이저 로드 (🥇 훈련된 모델 경로에서)
try:
    tokenizer = AutoTokenizer.from_pretrained(SAVED_MODEL_PATH)
    print(f"'{SAVED_MODEL_PATH}'에서 토크나이저 로드 완료.")
except OSError:
    print(f"오류: '{SAVED_MODEL_PATH}' 폴더를 찾을 수 없습니다. Input 경로를 확인하세요.")
    exit()

# 2. VADER 로드 (🏅 VADER 경로에서)
try:
    sia = joblib.load(VADER_PATH)
    print(f"'{VADER_PATH}' 로드 완료.")
except FileNotFoundError:
    print(f"오류: '{VADER_PATH}' 파일이 없습니다. Input 경로를 확인하세요.")
    exit()

# 3. 스케일러 로드 (🥇 훈련된 모델 경로에서)
try:
    scaler_path = os.path.join(SAVED_MODEL_PATH, "scaler.joblib")
    scaler = joblib.load(scaler_path)
    print(f"'{scaler_path}'에서 스케일러 로드 완료.")
except FileNotFoundError:
    print(f"오류: '{scaler_path}' 파일을 찾을 수 없습니다. Input 경로를 확인하세요.")
    exit()

# 4. 모델 로드 (PEFT + 커스텀 헤드)
print("--- [3] 모델 로드 시작 ---")
# 4-1. 원본(Base) DeBERTa 뼈대 로드 (🥈 원본 모델 경로에서)
base_backbone = AutoModelForSequenceClassification.from_pretrained(BASE_MODEL_PATH, num_labels=3)
print("  (a) 원본 DeBERTa 뼈대 로드 완료.")

# 4-2. 뼈대를 HybridClassifier 아키텍처로 감싸기
base_hybrid_model = HybridClassifier(base_backbone, bias_dim=BIAS_DIM)
print("  (b) HybridClassifier 아키텍처 구성 완료.")

# 4-3. 커스텀 헤드(model.safetensors) 가중치 수동 로드
head_weights_path = os.path.join(SAVED_MODEL_PATH, "model.safetensors")
try:
    head_weights = load_file(head_weights_path)
    base_hybrid_model.load_state_dict(head_weights, strict=False)
    print("  (c) 커스텀 헤드(model.safetensors) 가중치 수동 로드 완료.")
except FileNotFoundError:
    print(f"오류: '{head_weights_path}' 파일이 없습니다. Input 경로를 확인하세요.")
    exit()
except Exception as e:
    print(f"오류: 커스텀 헤드 로드 중 문제 발생: {e}")
    exit()


print("  (d) LoRA 어댑터 설정(config) 로드 중...")
try:
    config = PeftConfig.from_pretrained(SAVED_MODEL_PATH)
except Exception as e:
    print(f"오류: '{SAVED_MODEL_PATH}'에서 adapter_config.json 로드 실패: {e}")
    exit()


model = PeftModel(base_hybrid_model, config)
print("  (e) PeftModel 아키텍처 구성 완료.")


adapter_weights_path = os.path.join(SAVED_MODEL_PATH, "adapter_model.safetensors")
try:
    adapter_weights = load_file(adapter_weights_path)
    # strict=False
    model.load_state_dict(adapter_weights, strict=False) 
    print("  (f) LoRA 어댑터(adapter_model.safetensors) 가중치 수동 로드 완료.")
except FileNotFoundError:
    print(f"오류: '{adapter_weights_path}' 파일이 없습니다.")
    exit()
except Exception as e:
    print(f"오류: 어댑터 가중치 로드 중 문제 발생: {e}")
    exit()


model.to(device)
model.eval() 
print(f"--- [4] 저장된 모델 로드 완료 ({device}) ---")



print("--- [5] 테스트 데이터(test.csv) 추론 시작 ---")
try:
    test = pd.read_csv(TEST_CSV_PATH)
    print("test.csv 로드 완료.")
except FileNotFoundError:
    print(f"오류: '{TEST_CSV_PATH}' 파일이 없습니다. Input 경로를 확인하세요.")
    exit()

# 1. 훈련 때와 동일하게 피처 생성
test["a_sentiment"] = test["response_a"].apply(lambda x: sia.polarity_scores(str(x))["compound"])
test["b_sentiment"] = test["response_b"].apply(lambda x: sia.polarity_scores(str(x))["compound"])
test["a_len"] = test["response_a"].astype(str).apply(len)
test["b_len"] = test["response_b"].astype(str).apply(len)
test["a_word_cnt"] = test["response_a"].astype(str).apply(lambda x: len(x.split()))
test["b_word_cnt"] = test["response_b"].astype(str).apply(lambda x: len(x.split()))

a_style = test["response_a"].apply(style_counts).apply(pd.Series)
b_style = test["response_b"].apply(style_counts).apply(pd.Series)
a_style.columns = ["a_exclam", "a_qmark", "a_commas"]
b_style.columns = ["b_exclam", "b_qmark", "b_commas"]
test = pd.concat([test, a_style, b_style], axis=1)

for col in ["len", "word_cnt", "sentiment", "exclam", "qmark", "commas"]:
    test[f"diff_{col}"] = test[f"a_{col}"] - test[f"b_{col}"]

bias_feats_list = ["diff_len", "diff_word_cnt", "diff_sentiment", "diff_exclam", "diff_qmark", "diff_commas"]
test_bias_unscaled = test[bias_feats_list].values

# 2. 로드한 스케일러로 'transform'
test_bias_scaled = scaler.transform(test_bias_unscaled)
test_bias_tensor = torch.tensor(test_bias_scaled, dtype=torch.float).to(device)
print("테스트 피처 생성 및 스케일링 완료.")

# 3. 텍스트 피처 생성
sep = tokenizer.sep_token or "</s>"
test_texts = (test["prompt"] + f" {sep} " +
              test["response_a"] + f" {sep} " +
              test["response_b"]).tolist()

# 4. 토크나이징
encodings = tokenizer(
    test_texts, 
    padding=True, 
    truncation=True, 
    max_length=MAX_LEN, 
    return_tensors="pt"
)
encodings = {k: v.to(device) for k, v in encodings.items()}

# 5. 배치 추론
all_preds = []
print("배치 추론을 시작합니다...")
for i in range(0, len(test_texts), BATCH_SIZE):
    batch_input_ids = encodings["input_ids"][i:i+BATCH_SIZE]
    batch_attention_mask = encodings["attention_mask"][i:i+BATCH_SIZE]
    batch_bias_features = test_bias_tensor[i:i+BATCH_SIZE]
    
    with torch.no_grad():
        outputs = model(
            input_ids=batch_input_ids,
            attention_mask=batch_attention_mask,
            bias_features=batch_bias_features
        )
        
        logits = outputs["logits"]
        preds = torch.nn.functional.softmax(logits, dim=-1).cpu().numpy()
        all_preds.append(preds)

all_preds = np.concatenate(all_preds, axis=0)
print("배치 추론 완료.")

submission = pd.DataFrame({
    "id": test["id"],
    "winner_model_a": all_preds[:, 0],
    "winner_model_b": all_preds[:, 1],
    "winner_tie": all_preds[:, 2],
})

submission.to_csv("submission.csv", index=False)
print("--- [6] ✅ submission.csv 파일 생성 완료 ---")

# --- 메모리 정리 ---
del model, base_backbone, base_hybrid_model, tokenizer, test_bias_tensor, encodings, test, scaler, sia
torch.cuda.empty_cache()
gc.collect()

print("\n--- [7] 추론 스크립트 종료 ---")

2025-11-06 10:24:28.817891: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762424668.988172      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762424669.039269      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


--- [1] 라이브러리 임포트 완료 ---
--- [2] 훈련용 클래스/함수 재정의 완료 ---
'/kaggle/input/real-jonna-last/transformers/default/1'에서 토크나이저 로드 완료.
'/kaggle/input/vader/transformers/default/1/vader_model.pkl' 로드 완료.
'/kaggle/input/real-jonna-last/transformers/default/1/scaler.joblib'에서 스케일러 로드 완료.
--- [3] 모델 로드 시작 ---


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 1.7.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at /kaggle/input/debertav3small and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/peft/config.py:165: UserWarning: Unexpected keyword arguments ['target_parameters'] for class LoraConfig, these are ignored. This probably means that you're loading a configuration file that was saved using a higher version of th

  (a) 원본 DeBERTa 뼈대 로드 완료.
  (b) HybridClassifier 아키텍처 구성 완료.
  (c) 커스텀 헤드(model.safetensors) 가중치 수동 로드 완료.
  (d) LoRA 어댑터 설정(config) 로드 중...
  (e) PeftModel 아키텍처 구성 완료.
  (f) LoRA 어댑터(adapter_model.safetensors) 가중치 수동 로드 완료.
--- [4] 저장된 모델 로드 완료 (cuda) ---
--- [5] 테스트 데이터(test.csv) 추론 시작 ---
test.csv 로드 완료.
테스트 피처 생성 및 스케일링 완료.
배치 추론을 시작합니다...
배치 추론 완료.
--- [6] ✅ submission.csv 파일 생성 완료 ---

--- [7] 추론 스크립트 종료 ---
